# 🛠 Spring Buggy Simulator – Final Version with Full Slider Labels

**Features:**
- Start distance + platform midpoint goal
- Sliders for all physics variables (with labels shown fully)
- Simulate multiple trials with variance
- Verdict + energy comparison per trial
- Histogram of energy distribution

In [3]:
import ipywidgets as widgets
from IPython.display import display, Markdown, clear_output
import math
import matplotlib.pyplot as plt
import numpy as np

g = 9.81
ramp_length = 0.5
ramp_height = 0.2
ramp_theta = math.asin(ramp_height / ramp_length)

output_display = widgets.Output()
plot_display = widgets.Output()

def compute_required_energy(X, m, mu):
    E_flat = mu * m * g * X
    E_ramp_friction = mu * m * g * math.cos(ramp_theta) * ramp_length
    E_lift = m * g * ramp_height
    E_platform = mu * m * g * 0.25
    return E_flat + E_ramp_friction + E_lift + E_platform

def compute_available_energy(U, eff):
    return U * eff

def update_display(*args):
    m = mass_slider.value
    mu = friction_slider.value
    U = spring_energy_slider.value
    eff = efficiency_slider.value
    X = start_dist_slider.value
    I = inertia_slider.value
    r = radius_slider.value
    trials = trial_slider.value

    E_req = compute_required_energy(X, m, mu)
    base_energy = compute_available_energy(U, eff)
    E_rot = 0.5 * I * (1 / r)**2
    total_required = E_req + E_rot

    verdicts = []
    energies = []
    for _ in range(trials):
        noise = np.random.normal(0, 0.05 * base_energy)
        E_avail = base_energy + noise
        energies.append(E_avail)
        diff = E_avail - total_required
        if abs(diff) < 0.2:
            verdicts.append('✅ Hit')
        elif diff < 0:
            verdicts.append('⚠️ Undershoot')
        else:
            verdicts.append('❌ Overshoot')

    with output_display:
        clear_output()
        display(Markdown(f"### Required Total Energy: `{total_required:.2f}` J"))
        for i, (v, e) in enumerate(zip(verdicts, energies)):
            display(Markdown(f"Trial {i+1}: {v} – Energy: `{e:.2f}` J"))

    with plot_display:
        clear_output()
        plt.figure(figsize=(7, 2.5))
        plt.axvline(total_required, color='red', linestyle='--', label='Required')
        plt.hist(energies, bins=10, color='orange', edgecolor='black', alpha=0.7)
        plt.title('Distribution of Available Energy')
        plt.xlabel('Energy (J)')
        plt.ylabel('Trials')
        plt.legend()
        plt.tight_layout()
        plt.show()

In [4]:
# Define sliders
layout = widgets.Layout(width='700px')
style = {'handle_color': 'black'}

start_dist_slider = widgets.FloatSlider(min=2.0, max=4.0, value=3.0, step=0.1, layout=layout, style=style)
spring_energy_slider = widgets.FloatSlider(min=0.5, max=15.0, value=5.49, step=0.01, layout=layout, style=style)
mass_slider = widgets.FloatSlider(min=0.1, max=5.0, value=1.0, step=0.1, layout=layout, style=style)
friction_slider = widgets.FloatSlider(min=0.01, max=0.2, value=0.1, step=0.01, layout=layout, style=style)
efficiency_slider = widgets.FloatSlider(min=0.1, max=1.0, value=0.5, step=0.01, layout=layout, style=style)
inertia_slider = widgets.FloatSlider(min=0.0, max=0.05, value=0.01, step=0.001, layout=layout, style=style)
radius_slider = widgets.FloatSlider(min=0.01, max=0.1, value=0.03, step=0.005, layout=layout, style=style)
trial_slider = widgets.IntSlider(min=1, max=50, value=5, step=1, layout=layout, style=style)

# Observe changes
for s in [start_dist_slider, spring_energy_slider, mass_slider, friction_slider,
          efficiency_slider, inertia_slider, radius_slider, trial_slider]:
    s.observe(update_display, names='value')

# Display
update_display()
display(widgets.VBox([
    output_display,
    widgets.Label('Start Distance AB (m)'), start_dist_slider,
    widgets.Label('Spring Potential Energy (J)'), spring_energy_slider,
    widgets.Label('Buggy Mass (kg)'), mass_slider,
    widgets.Label('Friction Coefficient μ'), friction_slider,
    widgets.Label('Efficiency (%)'), efficiency_slider,
    widgets.Label('Rotational Inertia (kg·m²)'), inertia_slider,
    widgets.Label('Wheel Radius (m)'), radius_slider,
    widgets.Label('Number of Trials'), trial_slider,
    plot_display
]))